<a href="https://colab.research.google.com/github/cesaraugf/py-sql-github-data-project/blob/develop/enriquecimento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import duckdb

In [ ]:
con = duckdb.connect(database='dados_duckdb.db', read_only=False)

In [ ]:
df=con.execute("SELECT * FROM bronze_z0019").fetchdf()
df.head(10)

In [ ]:
df = con.execute("""
                SELECT * 
                FROM (
                    SELECT *, ROW_NUMBER() OVER (PARTITION BY NATBR ORDER BY data_ingestao DESC) as ROW
                    FROM bronze_z0019
                    WHERE data_ingestao >= '2025-10-15'
                 ) WHERE ROW = 1
                """).fetchdf()
df.head(10)

In [ ]:
df_final = df.drop(columns=['nome_arquivo', 'data_ingestao', 'ROW'])
df_final = df_final.rename(columns={'NATBR': 'id'})
df_final = df_final.rename(columns={'MAKTX': 'nm_produto'})
df_final = df_final.rename(columns={'WERKS': 'id_categoria'})
df_final = df_final.rename(columns={'MAINS': 'id_fornecedor'})
df_final = df_final.rename(columns={'LABST': 'vl_preco'})
df_final.head(10)

In [ ]:
con.commit()

In [ ]:
df2 = df_final
df2.head(10)

In [ ]:
df_final.dtypes

In [ ]:
df2 = df2.astype(
    {
        'id': int,
        'nm_produto': str,
        'id_categoria': str,
        'id_fornecedor': int,
        'vl_preco': float
    }
)
df2.dtypes

In [ ]:
con.execute("""
    CREATE TABLE IF NOT EXISTS produtos (
        id BIGINT,
        nm_produto TEXT,
        id_categoria TEXT,
        id_fornecedor BIGINT,
        vl_preco FLOAT
        )
""")

In [ ]:
df2.head(10)

In [ ]:
df_resultado = con.execute("SELECT * FROM produtos").fetchdf()

In [ ]:
df_resultado.head(10)

In [ ]:
con.execute("INSERT INTO produtos SELECT * FROM df2")

In [ ]:
con.commit()
con.close()